In [1]:
import os

def rename_subdirectories(directory_path):
    # Get the list of subdirectories
    subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

    # Rename each subdirectory by replacing spaces with underscores
    for subdirectory in subdirectories:
        new_name = subdirectory.replace(' ', '_')
        old_path = os.path.join(directory_path, subdirectory)
        new_path = os.path.join(directory_path, new_name)
        os.rename(old_path, new_path)
        print(f'Renamed: {subdirectory} -> {new_name}')

# Replace 'app/DashApp/HeartSound' with your actual directory path
directory_path = 'app/DashApp/HeartSound'
rename_subdirectories(directory_path)

Renamed: Aortic Valve Stenosis Regurgitation -> Aortic_Valve_Stenosis_Regurgitation
Renamed: Mitral Valve Prelapse -> Mitral_Valve_Prelapse
Renamed: Ebstein Anomaly -> Ebstein_Anomaly
Renamed: Graham Steell Murmur -> Graham_Steell_Murmur
Renamed: Holosystolic Murmur -> Holosystolic_Murmur
Renamed: Pericardial Rub -> Pericardial_Rub
Renamed: Acute Myocardial Infarction -> Acute_Myocardial_Infarction
Renamed: Dilated Cardiomyopathy -> Dilated_Cardiomyopathy
Renamed: Acute Pericarditis -> Acute_Pericarditis
Renamed: Ventricular Septal Defect -> Ventricular_Septal_Defect
Renamed: Normal Heart -> Normal_Heart
Renamed: Aortic Valve Stenosis -> Aortic_Valve_Stenosis
Renamed: Congenital Aortic Stenosis -> Congenital_Aortic_Stenosis
Renamed: Mitral Stenosis Tricuspid Regurgitation -> Mitral_Stenosis_Tricuspid_Regurgitation
Renamed: Aortic Valve Regurgitation -> Aortic_Valve_Regurgitation
Renamed: Patent Ductus Arteriosus -> Patent_Ductus_Arteriosus
Renamed: Pulmonary Hypertension -> Pulmonary_H

In [1]:
import os

def generate_import_statements(directory_path):
    # Get the list of subdirectories
    subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

    # Generate import statements
    for subdirectory in subdirectories:
        import_statement = f"from ..DashApp.HeartSound.{subdirectory} import aortic, erb, mitral, pulmonary, tricuspid"
        print(import_statement)

# Replace 'app/DashApp/HeartSound' with your actual directory path
directory_path = 'app/DashApp/HeartSound'
generate_import_statements(directory_path)

from ..DashApp.HeartSound.Pulmonary_Hypertension import aortic, erb, mitral, pulmonary, tricuspid
from ..DashApp.HeartSound.Patent_Ductus_Arteriosus import aortic, erb, mitral, pulmonary, tricuspid
from ..DashApp.HeartSound.Tetralogy_of_Fallot import aortic, erb, mitral, pulmonary, tricuspid
from ..DashApp.HeartSound.Hypertrophic_Cardiomyopathy import aortic, erb, mitral, pulmonary, tricuspid
from ..DashApp.HeartSound.Congestive_Heart_Failure import aortic, erb, mitral, pulmonary, tricuspid
from ..DashApp.HeartSound.Holosystolic_Murmur import aortic, erb, mitral, pulmonary, tricuspid
from ..DashApp.HeartSound.Graham_Steell_Murmur import aortic, erb, mitral, pulmonary, tricuspid
from ..DashApp.HeartSound.Tricuspid_Valve_Regurgitation import aortic, erb, mitral, pulmonary, tricuspid
from ..DashApp.HeartSound.Mitral_Valve_Stenosis import aortic, erb, mitral, pulmonary, tricuspid
from ..DashApp.HeartSound.Normal_Heart import aortic, erb, mitral, pulmonary, tricuspid
from ..DashApp.HeartSou